In [1]:
# Install required packages in the folllowing few cells

In [2]:
%%capture
 # capture hides output
!pip install yfinance pandas numpy scikit-learn ta pandas_ta xgboost

In [3]:
%%capture
# hidden output
!pip install alpha_vantage

In [4]:
%%capture
# hidden output, !pip install is required to run just on a machine
!pip install requests pandas_datareader


#### IMPORTS

In [21]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json
import pandas_ta as ta
from ta import trend, volatility, momentum, volume
import yfinance as yf
import time
from sklearn.model_selection import train_test_split


from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


In [22]:
data_yf = pd.read_csv('data/all_stock_data.csv')

In [23]:
data_yf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
0,2000-09-23,-0.278653,-0.276205,-0.286855,-0.287362,-0.307271,2.816789,MSFT,-0.188214,-0.199184,...,-1.275540,-0.621584,-0.410158,-6.111879,-6.109012,-6.106413,-6.104059,-6.101726,-1.013158,0
1,2000-09-30,-0.286880,-0.290906,-0.299534,-0.301516,-0.316185,2.689748,MSFT,-0.204746,-0.203402,...,-1.781449,-1.275530,-0.621576,-6.115239,-6.111671,-6.108804,-6.106205,-6.103852,-1.734271,0
2,2000-10-07,-0.301464,-0.302159,-0.306835,-0.306916,-0.319586,2.641980,MSFT,-0.219688,-0.209991,...,-1.587425,-1.781438,-1.275521,-6.118459,-6.115031,-6.111463,-6.108597,-6.105998,-0.752039,0
3,2000-10-14,-0.307822,-0.275479,-0.318170,-0.272836,-0.298124,4.791548,MSFT,-0.201616,-0.211375,...,-1.698473,-1.587415,-1.781428,-6.121626,-6.118251,-6.114824,-6.111256,-6.108390,4.418443,1
4,2000-10-21,-0.274539,-0.266586,-0.282052,-0.265387,-0.293432,4.498460,MSFT,-0.179198,-0.207657,...,-1.805761,-1.698463,-1.587405,-6.116081,-6.121418,-6.118043,-6.114616,-6.111049,0.910790,1


In [24]:
data_yf.isna().sum()


Date             0
Open             0
High             0
Low              0
Close            0
                ..
OBV_lag_3        0
OBV_lag_4        0
OBV_lag_5        0
gain_loss_pct    0
win              0
Length: 128, dtype: int64

In [25]:
data_yf.isna().any().any()

False

In [26]:
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
0,2000-09-23,-0.278653,-0.276205,-0.286855,-0.287362,-0.307271,2.816789,MSFT,-0.188214,-0.199184,...,-1.275540,-0.621584,-0.410158,-6.111879,-6.109012,-6.106413,-6.104059,-6.101726,-1.013158,0
1,2000-09-30,-0.286880,-0.290906,-0.299534,-0.301516,-0.316185,2.689748,MSFT,-0.204746,-0.203402,...,-1.781449,-1.275530,-0.621576,-6.115239,-6.111671,-6.108804,-6.106205,-6.103852,-1.734271,0
2,2000-10-07,-0.301464,-0.302159,-0.306835,-0.306916,-0.319586,2.641980,MSFT,-0.219688,-0.209991,...,-1.587425,-1.781438,-1.275521,-6.118459,-6.115031,-6.111463,-6.108597,-6.105998,-0.752039,0
3,2000-10-14,-0.307822,-0.275479,-0.318170,-0.272836,-0.298124,4.791548,MSFT,-0.201616,-0.211375,...,-1.698473,-1.587415,-1.781428,-6.121626,-6.118251,-6.114824,-6.111256,-6.108390,4.418443,1
4,2000-10-21,-0.274539,-0.266586,-0.282052,-0.265387,-0.293432,4.498460,MSFT,-0.179198,-0.207657,...,-1.805761,-1.698463,-1.587405,-6.116081,-6.121418,-6.118043,-6.114616,-6.111049,0.910790,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562609,2024-02-26,-0.302855,-0.304533,-0.299166,-0.303788,-0.254880,-0.198914,NWS,0.101111,0.095766,...,0.668709,1.106629,1.220597,1.215446,1.215403,1.215349,1.215282,1.215290,-0.194704,0
562610,2024-03-04,-0.303514,-0.306740,-0.303530,-0.304086,-0.255182,-0.199094,NWS,0.100358,0.098207,...,0.755070,0.668716,1.106635,1.215422,1.215420,1.215377,1.215324,1.215256,-0.150454,0
562611,2024-03-11,-0.304232,-0.305404,-0.305682,-0.309509,-0.260681,-0.186230,NWS,0.094285,0.098854,...,0.775355,0.755077,0.668723,1.215397,1.215396,1.215394,1.215351,1.215298,-0.743110,0
562612,2024-03-18,-0.308899,-0.309237,-0.305682,-0.307363,-0.257912,-0.198144,NWS,0.090169,0.098485,...,0.650980,0.775362,0.755084,1.215359,1.215372,1.215370,1.215369,1.215326,0.116609,1


In [27]:
first_appearance_dates = data_yf.groupby('Ticker')['Date'].min()
first_appearance_dates.head(20)

Ticker
A       2000-01-01
AAL     2005-09-26
AAPL    2000-01-01
ABBV    2013-01-01
ABNB    2020-12-07
ABT     2000-01-01
ACGL    2000-01-01
ACN     2001-07-16
ADBE    2000-01-01
ADI     2000-01-01
ADM     2000-01-01
ADP     2000-01-01
ADSK    2000-01-01
AEE     2000-01-01
AEP     2000-01-01
AES     2000-01-01
AFL     2000-01-01
AIG     2000-01-01
AIZ     2004-02-02
AJG     2000-01-01
Name: Date, dtype: object

In [28]:
# Initialize a list to collect DataFrames
#symbols = list(data_yf['Ticker'].value_counts().index)[0:300] #eases the number

symbols = list(data_yf['Ticker'].value_counts().index)


df_list = []

# Loop over each unique ticker
for ticker in range(len(symbols)):
    df = data_yf[data_yf['Ticker'] == symbols[ticker]]
    
    # Rename all columns in df to "original_column_nameTicker"
    df.columns = [str(col) + str(symbols[ticker]) for col in df.columns]
    
    df = df.reset_index().drop(columns=['index'])#[0:990]
    
    # Append modified df to the list
    df_list.append(df)



In [29]:
num_null = 0

for df in df_list:
    if df.isna().any().any() == True:
        num_null +=1 
        
data = pd.concat(df_list, axis=1)


data.set_index('DateFITB', inplace=True)

In [30]:
date_values = [col for col in data.columns if 'Date' in col]
data.drop(columns=date_values, inplace=True)

In [31]:
data.isna().sum()

OpenEXC                 0
HighEXC                 0
LowEXC                  0
CloseEXC                0
Adj CloseEXC            0
                     ... 
OBV_lag_3VLTO        1240
OBV_lag_4VLTO        1240
OBV_lag_5VLTO        1240
gain_loss_pctVLTO    1240
winVLTO              1240
Length: 60878, dtype: int64

In [32]:
data.shape

(1266, 60878)

Now, we will do Predict then Optimize -- Shared Learning

<h2> Predict then Optimize -- Shared Learning</h2>

In [33]:
# from sklearn.preprocessing import StandardScaler

float_columns = data.select_dtypes(include=['float64']).columns

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Standardize only the float columns
# data[float_columns] = scaler.fit_transform(data[float_columns])

In [34]:
data = data[float_columns]

data.fillna(-99, inplace=True)

In [35]:
data.shape

(1266, 57529)

In [57]:
data.head()

,OpenEXC,HighEXC,LowEXC,CloseEXC,Adj CloseEXC,VolumeEXC,macdEXC,macd_signalEXC,rsiEXC,bb_highEXC,...,WILLR_14_lag_3VLTO,WILLR_14_lag_4VLTO,WILLR_14_lag_5VLTO,OBV_lag_1VLTO,OBV_lag_2VLTO,OBV_lag_3VLTO,OBV_lag_4VLTO,OBV_lag_5VLTO,gain_loss_pctVLTO,winVLTO
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,-0.394259,-0.390087,-0.394707,-0.392025,-0.388661,-0.159118,-0.182307,0.324427,-2.622972,0.706015,...,-0.388159,-0.114209,0.194316,0.615216,0.615221,0.615147,0.615238,0.615362,0.289459,1.0
2000-01-08,-0.389324,-0.386722,-0.386897,-0.385251,-0.385791,-0.167663,-0.831001,0.080735,-2.592330,0.786047,...,0.111852,-0.388150,-0.114201,0.615455,0.615209,0.615214,0.615140,0.615232,-1.344911,0.0
2000-01-15,-0.386257,-0.386592,-0.385389,-0.384454,-0.385454,-0.158123,-1.329128,-0.221401,-2.588469,0.834990,...,-0.124855,0.111860,-0.388142,0.615250,0.615449,0.615203,0.615208,0.615134,-0.544719,0.0
2000-01-22,-0.384923,-0.379472,-0.382101,-0.379539,-0.383372,-0.034487,-1.700319,-0.542979,-2.562949,0.866802,...,1.098229,-0.124847,0.111868,0.615128,0.615244,0.615442,0.615196,0.615201,-0.448484,0.0
2000-01-29,-0.379855,-0.377013,-0.378127,-0.375687,-0.381741,-0.082007,-1.968678,-0.857985,-2.541568,0.883917,...,0.381055,1.098235,-0.124839,0.614993,0.615121,0.615237,0.615436,0.615190,0.633771,1.0


In [36]:
import tensorflow as tf
from scipy.optimize import minimize

In [47]:
device_name = "GPU" if tf.config.experimental.list_physical_devices('GPU') else "cpu"
print("Using device:", device_name)

Using device: GPU


In [48]:
physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [76]:
# Prepare features and targets for model training
def prepare_features_targets(data_2):
#     indicators = calculate_technical_indicators(data)
#     returns = np.log(data / data.shift(1)).add_suffix('_returns')
#     full_data = pd.concat([indicators, returns], axis=1).dropna()
    features = data_2[[col for col in data_2.columns if 'gain' not in col]]
    targets = data_2[[col for col in data_2.columns if 'gain' in col]]
    return features, targets

# Build and compile a neural network
def build_model(input_shape, output_shape):
    with tf.device(device_name):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(output_shape)
        ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Portfolio optimization function with variance consideration and risk aversion
def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
    weights = weights / np.sum(weights)  # Normalize weights
    pred_returns = model.predict(np.array([features.iloc[-1]]))[0]
    expected_return = np.dot(weights, pred_returns)
    covariance_matrix = historical_returns.cov()
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    # Objective function with risk aversion parameter
    return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance


In [60]:
#set index to datetime
data.index = pd.to_datetime(data.index)

In [87]:
data.head()

,OpenEXC,HighEXC,LowEXC,CloseEXC,Adj CloseEXC,VolumeEXC,macdEXC,macd_signalEXC,rsiEXC,bb_highEXC,...,WILLR_14_lag_3VLTO,WILLR_14_lag_4VLTO,WILLR_14_lag_5VLTO,OBV_lag_1VLTO,OBV_lag_2VLTO,OBV_lag_3VLTO,OBV_lag_4VLTO,OBV_lag_5VLTO,gain_loss_pctVLTO,winVLTO
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,-0.394259,-0.390087,-0.394707,-0.392025,-0.388661,-0.159118,-0.182307,0.324427,-2.622972,0.706015,...,-0.388159,-0.114209,0.194316,0.615216,0.615221,0.615147,0.615238,0.615362,0.289459,1.0
2000-01-08,-0.389324,-0.386722,-0.386897,-0.385251,-0.385791,-0.167663,-0.831001,0.080735,-2.592330,0.786047,...,0.111852,-0.388150,-0.114201,0.615455,0.615209,0.615214,0.615140,0.615232,-1.344911,0.0
2000-01-15,-0.386257,-0.386592,-0.385389,-0.384454,-0.385454,-0.158123,-1.329128,-0.221401,-2.588469,0.834990,...,-0.124855,0.111860,-0.388142,0.615250,0.615449,0.615203,0.615208,0.615134,-0.544719,0.0
2000-01-22,-0.384923,-0.379472,-0.382101,-0.379539,-0.383372,-0.034487,-1.700319,-0.542979,-2.562949,0.866802,...,1.098229,-0.124847,0.111868,0.615128,0.615244,0.615442,0.615196,0.615201,-0.448484,0.0
2000-01-29,-0.379855,-0.377013,-0.378127,-0.375687,-0.381741,-0.082007,-1.968678,-0.857985,-2.541568,0.883917,...,0.381055,1.098235,-0.124839,0.614993,0.615121,0.615237,0.615436,0.615190,0.633771,1.0


In [88]:
data.shape

(1266, 57529)

In [89]:
# Main script
#features, targets = prepare_features_targets(data)
with tf.device(device_name):
    print(device_name)
    # Your model code here
    split_date = pd.Timestamp('2022-01-01')
    # Prepare your features and targets first
    features, targets = prepare_features_targets(data)
    # Split the data based on the threshold date
    train_features = features[features.index < split_date]
    test_features = features[features.index >= split_date]
    train_targets = targets[targets.index < split_date]
    test_targets = targets[targets.index >= split_date]
    #features, targets = prepare_features_targets(data)
    #train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)
    model = build_model(train_features.shape[1], train_targets.shape[1])
    model.fit(train_features, train_targets, epochs=3, batch_size=5)


# train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# model = build_model(train_features.shape[1], train_targets.shape[1])
# model.fit(train_features, train_targets, epochs=3, batch_size=5)

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
initial_weights = np.ones(len(symbols)) / len(symbols)  # Adjust this based on the actual number of features or stocks.
bounds = tuple((0, 1) for _ in symbols)


# optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.999999999999999999999999), bounds=bounds, constraints=constraints)
# optimized_weights = optimized_result.x

with tf.device(device_name):
    # Optimizing the portfolio on training data
    train_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
    optimized_weights_train = train_optimized_result.x

    # Optimizing the portfolio on testing data
    test_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
    optimized_weights_test = test_optimized_result.x

    # Calculating ROI
    train_predicted_returns = model.predict(train_features)
    train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

    test_predicted_returns = model.predict(test_features)
    test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment


GPU
Epoch 1/3
230/230 [==============================] - 3s 11ms/step - loss: 46.3236
Epoch 2/3
230/230 [==============================] - 3s 11ms/step - loss: 14.1130
Epoch 3/3
4/4 [==============================] - 0s 11ms/step


In [90]:
print("Optimized Portfolio Weights (Train):", optimized_weights_train)
print("Training ROI:", train_roi)
print("Optimized Portfolio Weights (Test):", optimized_weights_test)
print("Testing ROI:", test_roi)

Optimized Portfolio Weights (Train): [1.13446627e-11 0.00000000e+00 1.71083879e-12 5.71996164e-12
 0.00000000e+00 2.09639986e-12 0.00000000e+00 0.00000000e+00
 1.83557619e-12 5.80504869e-12 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.37248254e-12
 2.37771695e-13 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.61006546e-12
 0.00000000e+00 2.40833174e-12 0.00000000e+00 0.00000000e+00
 1.77482523e-12 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 7.50649337e-01 7.57606322e-12 0.00000000e+00 2.71988294e-12
 0.00000000e+00 0.00000000e+00 2.70764162e-12 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.61657646e-12 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.94836036e-12 0.00000000e+00 0.00000000e+00
 2.36403358e-12 0.00000000e+00 0.00000000e+00 0.

In [96]:
def print_non_zero_weights(weights, title):
    # Using a threshold to consider weights effectively zero
    threshold = 1e-10
    non_zero_weights = {index: weight for index, weight in enumerate(weights) if abs(weight) > threshold}
    print(f"{title} (non-zero weights):")
    for index, weight in non_zero_weights.items():
        print(f"Index {index}: Weight {weight}")

# Assuming optimized_weights_train and optimized_weights_test are your weight arrays
print_non_zero_weights(optimized_weights_train, "Optimized Portfolio Weights (Train)")
print("Training ROI:", train_roi)
print_non_zero_weights(optimized_weights_test, "Optimized Portfolio Weights (Test)")
print("Testing ROI:", test_roi)


Optimized Portfolio Weights (Train) (non-zero weights):
Index 36: Weight 0.7506493373053312
Index 107: Weight 0.24935066301759162
Training ROI: 13449.516578751605
Optimized Portfolio Weights (Test) (non-zero weights):
Index 240: Weight 1.0
Testing ROI: 25724.341897955263


In [97]:

print("Training ROI:", train_roi)

print("Testing ROI:", test_roi)

Training ROI: 13449.516578751605
Testing ROI: 25724.341897955263


In [98]:
max(optimized_weights_train)

0.7506493373053312

In [99]:
train_predicted_returns

array([[  7.367581  , -26.090761  ,  -4.5697474 , ...,  -5.5588455 ,
         17.96151   ,  23.50161   ],
       [-12.812023  , -22.83175   , -14.081818  , ..., -10.618287  ,
         15.21591   ,  14.542645  ],
       [-21.976814  , -31.86277   , -12.097797  , ...,   4.160568  ,
         11.928576  ,  11.017948  ],
       ...,
       [  4.4118304 ,   1.8443525 ,   2.367479  , ...,  -2.466879  ,
          0.53773296,  -1.9416368 ],
       [  4.677593  ,   2.2087169 ,   2.208722  , ...,  -2.725632  ,
         -0.25632423,  -1.7570567 ],
       [  4.606506  ,   2.1923413 ,   2.3465662 , ...,  -1.74674   ,
         -1.0271655 ,  -2.0614517 ]], dtype=float32)

In [100]:
# Assuming 'data' is indexed by datetime
yearly_splits = pd.date_range(start='2000-01-01', end='2024-01-01', freq='Y')

annual_weights = {}
annual_returns = {}

for start, end in zip(yearly_splits[:-1], yearly_splits[1:]):
    year_data = data[start:end]
    features, targets = prepare_features_targets(year_data)
    
    # Split into training and testing data if necessary or use the full year for training
    train_features, train_targets = features, targets
    
    # Initialize and train model
    model = build_model(train_features.shape[1], train_targets.shape[1])
    model.fit(train_features, train_targets, epochs=3, batch_size=5)
    
    # Optimize portfolio
    result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
    annual_weights[start.year] = result.x
    
    # Simulate or calculate the ROI for the year
    predicted_returns = model.predict(train_features)
    annual_returns[start.year] = np.dot(result.x, predicted_returns.mean(axis=0)) * 10000  # Modify as per actual investment

# Display the results
print("Annual Weights:", annual_weights)
print("Annual Returns:", annual_returns)


Epoch 1/3
11/11 [==============================] - 0s 17ms/step - loss: 223.3198
Epoch 2/3
11/11 [==============================] - 0s 13ms/step - loss: 258.9197
Epoch 3/3
2/2 [==============================] - 0s 60ms/step
Epoch 1/3
11/11 [==============================] - 0s 14ms/step - loss: 171.9920
Epoch 2/3
11/11 [==============================] - 0s 9ms/step - loss: 224.2952
Epoch 3/3
2/2 [==============================] - 0s 16ms/step
Epoch 1/3
11/11 [==============================] - 0s 16ms/step - loss: 281.9626
Epoch 2/3
11/11 [==============================] - 0s 11ms/step - loss: 289.6401
Epoch 3/3
2/2 [==============================] - 0s 27ms/step
Epoch 1/3
11/11 [==============================] - 0s 16ms/step - loss: 114.9616
Epoch 2/3
11/11 [==============================] - 0s 11ms/step - loss: 170.9121
Epoch 3/3
2/2 [==============================] - 0s 17ms/step
Epoch 1/3
11/11 [==============================] - 0s 15ms/step - loss: 183.6307
Epoch 2/3
11/11 [=======

In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from scipy.optimize import minimize
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # Prepare features and targets for model training
# def prepare_features_targets(data):
#     features = data[[col for col in data.columns if 'gain' not in col]]
#     targets = data[[col for col in data.columns if 'gain' in col]]
#     return features, targets

# # Build and compile a neural network
# def build_model(input_shape, output_shape):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(output_shape)
#     ])
#     model.compile(optimizer='adam', loss='mse')
#     return model

# # Portfolio optimization function with variance consideration and risk aversion
# def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
#     weights = weights / np.sum(weights)  # Normalize weights
#     pred_returns = model.predict(features)[-1]  # Predicting the last set of features
#     expected_return = np.dot(weights, pred_returns)
#     covariance_matrix = historical_returns.cov()
#     portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
#     return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance

# column_values = [col for col in data.columns if 'EXC' or 'APPL' in col]
# data_2 = data[column_values]

# symbols = ['EXC', 'AAPL']

# # Prepare features and targets
# features, targets = prepare_features_targets(data_2)

# # Split data into training and testing sets
# train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# # Build model
# model = build_model(train_features.shape[1], train_targets.shape[1])

# # Train model
# model.fit(train_features, train_targets, epochs=3, batch_size=32)

# # Optimizing portfolio using training data
# # train_returns = train_features.pct_change().dropna()
# initial_weights = np.ones(len(symbols)) / len(symbols)
# bounds = tuple((0, 1) for _ in symbols)
# constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

# optimized_result_train = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
# optimized_weights_train = optimized_result_train.x

# # Calculate ROI for the training set
# train_predicted_returns = model.predict(train_features)
# train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# # Optimizing portfolio using testing data
# test_returns = test_features.pct_change().dropna()
# optimized_result_test = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
# optimized_weights_test = optimized_result_test.x

# # Calculate ROI for the testing set
# test_predicted_returns = model.predict(test_features)
# test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# print(f"Training ROI: ${train_roi:.2f}")
# print(f"Testing ROI: ${test_roi:.2f}")
# print("Optimized Weights for Training Set:", optimized_weights_train)
# print("Optimized Weights for Testing Set:", optimized_weights_test)


Non Shared Learning


In [ ]:
data_yf

In [ ]:
#Need a train loader and a test loader
#Need to convert to optDataset, then wrap with DataLoader

In [ ]:
!pip install PyPl gurobipy pyepo

Create a LSTM-based model for stock prediction tasks

In [ ]:
import torch.nn.functional as F
from torch import nn
import torch 
class StockLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(StockLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        all_h, (h, c) = self.lstm(x)
        out = self.fc(all_h) # Apply Linear layer to outputs from all the hidden state.
        return out

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader

m = 50 # change based on number of assets
cov = np.cov(np.random.randn(10, m), rowvar=False) # covariance matrix
optmodel = pyepo.model.grb.portfolioModel(m, cov) # build model

Auto-Sklearn cannot be imported.
Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    train_loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        train_loss = 0
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            elif method_name == "mse":
                loss = loss_func(cp, c)
            else:
                raise ValueError("Method name {} not supported".format(method_name))
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            train_loss += loss.item()
        train_loss /= len(loader_train)
        train_loss_log.append(train_loss)
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, train_loss, regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return train_loss_log, loss_log_regret

Create a Predict-then-Optimize Model

In [ ]:

spop = pyepo.func.SPOPlus(optmodel, processes=1)

In [ ]:
# Hyperparameters
VOCAB_SIZE = #fill in once data is imported
EMBEDDING_DIM = #param we can optimize
HIDDEN_DIM = #also a param to optimize
learning_rate = #another optimizable param
epoch = #optimize
num_layers = #optimize
epochs = 20
learning_rate = 2e-3
method_name = "spo+"

# Instantiate the model
lstm = StockLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,num_layers=num_layers)
loss_log_lstm_spo, loss_log_regret_lstm_spo = trainModel(lstm, loss_func=spop, method_name=method_name)
